# Silver Data Transformation

### PULL THE LIBRARISE

In [0]:
#Pull the libraries 
from pyspark.sql.functions import *
from pyspark.sql.types import *


In [0]:
df = spark.read.format("delta")\
    .option("header", True)\
    .option("inferSchema", True)\
    .load("abfss://bronze@netflixprojectdlpams.dfs.core.windows.net/netflix_titles")


In [0]:
display(df)

#### GIT RID OF NULLS

In [0]:
df = df.fillna({"duration_minutes": 0 , "duration_seasons": 1})

In [0]:
df.display()

In [0]:
df.describe().display()


In [0]:
df.withColumn("duration_minutes", df["duration_minutes"].cast(IntegerType()))

DataFrame[duration_minutes: int, duration_seasons: string, type: string, title: string, date_added: string, release_year: string, rating: string, description: string, show_id: string, _rescued_data: string]

In [0]:
df.display()

### CONVERT DATA TYPES

In [0]:
df = df.withColumn("duration_minutes", col("duration_minutes").cast(IntegerType()))\
    .withColumn("duration_seasons", col("duration_seasons").cast(IntegerType()))\
    .withColumn("release_year", col("release_year").cast(IntegerType()))\
    .withColumn("date_added" , to_date(col("date_added"), "M/d/yyyy"))\
    .withColumn("type", lower(trim(col("type"))))\
    .withColumn("title", lower(trim(col("title"))))\
    .withColumn("rating", lower(trim(col("rating"))))

In [0]:
df.printSchema()

root
 |-- duration_minutes: integer (nullable = true)
 |-- duration_seasons: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- date_added: date (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- rating: string (nullable = true)
 |-- description: string (nullable = true)
 |-- show_id: string (nullable = true)
 |-- _rescued_data: string (nullable = true)



#### DROP ROWS WITH MISSING CRITICAL VALUES

In [0]:
df = df.dropna(subset=["title", "type", "release_year"])

In [0]:
df.describe().display()

In [0]:
df.display(5)

In [0]:
df.select("rating").distinct().show()

+--------+
|  rating|
+--------+
|   tv-14|
|       g|
|      ur|
|   tv-pg|
|    NULL|
|tv-y7-fv|
|      nr|
|   pg-13|
|   nc-17|
|      pg|
|       r|
|   tv-ma|
|    tv-y|
|    tv-g|
|   tv-y7|
+--------+



In [0]:
# See the count 
df.groupBy("rating").count().orderBy("count", ascending=False).show()

+--------+-----+
|  rating|count|
+--------+-----+
|   tv-ma| 2027|
|   tv-14| 1698|
|   tv-pg|  700|
|       r|  508|
|   pg-13|  286|
|      nr|  218|
|      pg|  184|
|   tv-y7|  169|
|    tv-g|  149|
|    tv-y|  143|
|tv-y7-fv|   95|
|       g|   37|
|    NULL|   11|
|      ur|    7|
|   nc-17|    2|
+--------+-----+



In [0]:
# See the count 
movies_count_release_year = df.groupBy("release_year").count().orderBy("count", ascending=False)

In [0]:
movies_count_release_year.display()

release_year,count
2018,1063
2017,959
2019,843
2016,830
2015,517
2014,288
2013,237
2012,183
2010,149
2011,136


### REMOVE DUPLICATES

In [0]:
df = df.dropDuplicates()

In [0]:
df = df.fillna({"rating": "unknown"})

### DROP UNNECESSORY COLUMN

In [0]:
df = df.drop("_rescued_data")

In [0]:
df.display(5)

### CLEAN WHITESPACE 

In [0]:
df = df.withColumn("description",trim(col("description")))

In [0]:
df.display(5)

### WHEN OTHERWISE

In [0]:
df = df.withColumn("type_flag", when(col("type") == "movie", 1)\
                                .when(col("type") == "tv show", 2)\
                                .otherwise(0))
display(df)

### SQL QUERIES

In [0]:
# this can be used only on this notebook not for global so we can say this view is only for local this notebook 
df_sql = df.createOrReplaceTempView("temp_view")

In [0]:
df_sql = spark.sql("""
               SELECT * FROM temp_view
               """)

In [0]:
df_sql.display()

duration_minutes,duration_seasons,type,title,date_added,release_year,rating,description,show_id,type_flag
0,1,tv show,forget me not,2017-09-18,2011,tv-ma,"Teenage Xue Ming was forced to marry the man who raped her. Years after abandoning her young sons to find a better life, can she escape the guilt?",80136785,2
97,1,movie,the kindergarten teacher,2018-10-12,2018,r,A devoted teacher takes interest in a young student's creative potential after hearing his poetry. But she soon crosses the line into obsession.,80240972,1
96,1,movie,walking out,2019-10-01,2017,pg-13,"A suburban teen goes hunting with his estranged father. When a mishap with a bear occurs, he must his use his survival skills to save them both.",80173150,1
129,1,movie,luv shuv tey chicken khurana,2018-10-01,2012,tv-14,"Fleeing London to evade a gangland debt collector, Omi Khurana returns to his native village in India, passing himself off as a successful lawyer.",70229042,1
129,1,movie,phantom,2018-10-01,2015,tv-14,India's primary intelligence agency sends an enigmatic operative on a top-secret mission to avenge the 11/26 terrorist attack on Mumbai.,70303495,1
107,1,movie,i am bolt,2017-10-01,2016,pg,"Mixing archival footage with present-day interviews, Olympic champion and global icon Usain Bolt opens up about his athletic legacy.",80135360,1
112,1,movie,christmas in the heartland,2019-11-01,2017,tv-pg,Two girls realize they're both visiting grandparents they’ve never met and decide to switch places to see how the other half lives.,81063129,1
0,1,tv show,kiss the series,2019-05-26,2016,tv-14,"After a night out, Sandee wakes to discover she and her buddy have either taken their platonic friendship to a whole new level or ruined it forever.",80993756,2
93,1,movie,russell madness,2015-05-10,2015,pg,"A spunky terrier named Russell with serious wrestling chops becomes a sports phenomenon with the help of his coach, a monkey named Hunk.",80028359,1
128,1,movie,starting over again,2019-03-07,2014,tv-14,"An architecture student and a history professor fall in love, pursue a dream, split up and bump into each other years later. Can there be a second chance?",81010865,1


In [0]:
# we can use global views 
df_sql.createOrReplaceGlobalTempView("global_view")


In [0]:
df_sql = spark.sql("""
               SELECT * FROM global_temp.global_view
               """)
               

In [0]:
df_sql = spark.sql("""
          SELECT rating, COUNT(*) AS count
          FROM global_temp.global_view
          GROUP BY rating
          ORDER BY count DESC 
          """).display()

rating,count
tv-ma,2027
tv-14,1698
tv-pg,700
r,508
pg-13,286
nr,218
pg,184
tv-y7,169
tv-g,149
tv-y,143


Databricks visualization. Run in Databricks to view.

In [0]:
print(df)

DataFrame[duration_minutes: int, duration_seasons: int, type: string, title: string, date_added: date, release_year: int, rating: string, description: string, show_id: string, type_flag: int]


In [0]:
df.write.format("delta")\
    .mode("overwrite")\
    .option("path", "abfs://silver@netflixprojectdlpams.dfs.core.windows.net/netflix_titles")\
    .save()